In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import ccxt
import os 

In [3]:
# Prepare data
start = '2019-01-01 UTC'  # crypto is in UTC
end = datetime.utcnow()

if os.path.exists('/root/trading_systems/btc_price_1d.pkl'):
    btc_price = pd.read_pickle('/root/trading_systems/btc_price_1d.pkl')
    print('File exitst already')
else:
    btc_price = vbt.CCXTData.download(exchange = 'kucoin',symbols = 'BTC/USDT',timeframe='1d', start=start, end=end).get()
    btc_price = pd.read_pickle('/root/trading_systems/btc_price_1d.pkl')
    print(btc_price)
    
btc_price.head()

# eth_price = vbt.YFData.download('ETH-USD', start=start, end=end).get('Close')
# comb_price = btc_price.vbt.concat(eth_price,
#     keys=pd.Index(['BTC', 'ETH'], name='symbol'))
# comb_price.vbt.drop_levels(-1, inplace=True)
# comb_price

File exitst already


,Open,High,Low,Close,Volume
Open time,,,,,
2019-01-01 00:00:00+00:00,3700.170853,3805.841739,3645.525763,3799.644687,97.900396
2019-01-02 00:00:00+00:00,3799.644687,3880.257733,3750.000001,3856.504318,159.232564
2019-01-03 00:00:00+00:00,3857.633097,3862.600830,3728.917225,3765.999334,110.163702
2019-01-04 00:00:00+00:00,3767.854300,3821.195343,3706.000000,3791.461260,92.251254
2019-01-05 00:00:00+00:00,3789.000000,3838.674540,3756.410862,3772.156039,93.872993


## calculte indicators

In [4]:
# calculate emas 
# ema from 1 to 6 1 shortest 6 longest 
ema1 = vbt.MA.run(btc_price['Close'], window=20)
ema2 = vbt.MA.run(btc_price['Close'], window=40)
ema3 = vbt.MA.run(btc_price['Close'], window=70)
ema4 = vbt.MA.run(btc_price['Close'], window=100)
ema5 = vbt.MA.run(btc_price['Close'], window=125)
ema6 = vbt.MA.run(btc_price['Close'], window=150)

rsi = vbt.RSI.run(btc_price['Close'], window=14).rsi
rsi.info()



<class 'pandas.core.series.Series'>
DatetimeIndex: 2174 entries, 2019-01-01 00:00:00+00:00 to 2024-12-13 00:00:00+00:00
Freq: D
Series name: (14, 'Close')
Non-Null Count  Dtype  
--------------  -----  
2160 non-null   float64
dtypes: float64(1)
memory usage: 34.0 KB


## ma cross slow 

In [19]:

def ma_cross_slow_indicator(close_price,window_slow,window_mid,window_fast):
    ema4 = vbt.MA.run(close_price, window=window_slow).ma.to_numpy()
    ema5 = vbt.MA.run(close_price, window=window_mid).ma.to_numpy()
    ema6 = vbt.MA.run(close_price, window=window_fast).ma.to_numpy()
    up = (ema4 > ema5) & (ema5 > ema6)
    down = (ema4 < ema5) & (ema5 < ema6)
    trend = np.where(up,1,np.where(down,-1,0))

    return trend


ma_cross_slow = vbt.IndicatorFactory(class_name='MA_Cross_slow',
                                     short_name='ma_c_slow',
                                     input_names=['Close'],
                                     param_names=['ema4', 'ema5', 'ema6'],
                                     output_names=['ma_cross_slow']
                                     ).from_apply_func(ma_cross_slow_indicator,ema4= 20, ema5=40,ema6=70)

window_slow = [180,200]
window_mid = [40,50]
window_fast = [20,30]

trend = ma_cross_slow.run(btc_price['Close'],ema4= window_fast, ema5=window_mid,ema6=window_slow,param_product=True)
trend.ma_cross_slow




ma_c_slow_ema4             20              30            
ma_c_slow_ema5             40      50      40      50    
ma_c_slow_ema6            180 200 180 200 180 200 180 200
Open time                                                
2019-01-01 00:00:00+00:00   0   0   0   0   0   0   0   0
2019-01-02 00:00:00+00:00   0   0   0   0   0   0   0   0
2019-01-03 00:00:00+00:00   0   0   0   0   0   0   0   0
2019-01-04 00:00:00+00:00   0   0   0   0   0   0   0   0
2019-01-05 00:00:00+00:00   0   0   0   0   0   0   0   0
...                        ..  ..  ..  ..  ..  ..  ..  ..
2024-12-09 00:00:00+00:00   1   1   1   1   1   1   1   1
2024-12-10 00:00:00+00:00   1   1   1   1   1   1   1   1
2024-12-11 00:00:00+00:00   1   1   1   1   1   1   1   1
2024-12-12 00:00:00+00:00   1   1   1   1   1   1   1   1
2024-12-13 00:00:00+00:00   1   1   1   1   1   1   1   1

[2174 rows x 8 columns]

In [10]:
entries = trend.ma_cross_slow == 'up'
exits = trend.ma_cross_slow == 'undecided'
short_entries = trend.ma_cross_slow == 'down'
short_exits = trend.ma_cross_slow == 'undecided'

pf = vbt.Portfolio.from_signals(
    btc_price['Close'], 
    entries, 
    exits, 
    short_entries=short_entries,
    short_exits=short_exits,
    open=btc_price['Open'],
    high=btc_price['High'], 
    low= btc_price['Low'],
    init_cash=1000, 
    fees=0.001,

) 


# pf.plot().show()
pf.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,"(20, 40, 180)",0.084000,2019-06-29 00:00:00+00:00,11892.9,0.999001,2019-07-26 00:00:00+00:00,9832.0,0.825886,-174.940036,-0.175115,Long,Closed,0
1,1,"(20, 40, 180)",0.080072,2019-08-15 00:00:00+00:00,10293.7,0.824236,2019-08-29 00:00:00+00:00,9493.5,0.760162,-65.657902,-0.079659,Long,Closed,1
2,2,"(20, 40, 180)",0.074597,2019-09-20 00:00:00+00:00,10169.9,0.758643,2019-09-24 00:00:00+00:00,8503.8,0.634357,-125.678962,-0.165663,Long,Closed,2
3,3,"(20, 40, 180)",0.077578,2019-10-15 00:00:00+00:00,8160.7,0.633090,2019-10-31 00:00:00+00:00,9141.6,0.709186,-77.438443,-0.122318,Short,Closed,3
4,4,"(20, 40, 180)",0.076015,2019-11-23 00:00:00+00:00,7310.8,0.555729,2020-01-07 00:00:00+00:00,8150.2,0.619536,-64.982079,-0.116931,Short,Closed,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,253,"(30, 50, 200)",0.064794,2024-05-30 00:00:00+00:00,68353.0,4.428896,2024-06-28 00:00:00+00:00,60439.2,3.916125,-521.115426,-0.117663,Long,Closed,253
254,254,"(30, 50, 200)",0.064370,2024-08-03 00:00:00+00:00,60716.3,3.908301,2024-08-10 00:00:00+00:00,60922.9,3.921600,5.468917,0.001399,Long,Closed,254
255,255,"(30, 50, 200)",0.064818,2024-08-22 00:00:00+00:00,60381.1,3.913764,2024-09-19 00:00:00+00:00,62950.4,4.080301,-174.530198,-0.044594,Short,Closed,255
256,256,"(30, 50, 200)",0.059045,2024-09-23 00:00:00+00:00,63331.4,3.739409,2024-09-29 00:00:00+00:00,65598.0,3.873240,-141.444262,-0.037825,Short,Closed,256


In [9]:
btc_price['Close']

ema4 = vbt.MA.run(btc_price['Close'], window=window_slow).ma
ema5 = vbt.MA.run(btc_price['Close'], window=window_mid).ma
ema6 = vbt.MA.run(btc_price['Close'], window=window_fast).ma


fig = btc_price['Close'].vbt.plot()
fig = ema4.vbt.plot(trace_kwargs = dict(name= 'ema_fast',line= dict(color='yellow' )),fig =fig)
fig = ema5.vbt.plot(trace_kwargs=dict(name='ema_mid'),fig =fig)
fig = ema6.vbt.plot(fig =fig)
fig = pf.trades.plot(fig =fig)
fig.show()



TypeError: Only one column is allowed. Use indexing or column argument.

In [80]:
mult_comb_price, _ = vbt.range_split(n=2)
mult_comb_price
fast_ma = vbt.MA.run(mult_comb_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(mult_comb_price, [30, 30], short_name='slow')

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(mult_comb_price, entries, exits, freq='1D')
pf.total_return()

AttributeError: module 'vectorbt' has no attribute 'range_split'

In [78]:
mean_return = pf.total_return().groupby(['split_idx', 'symbol']).mean()
mean_return.unstack(level=-1).vbt.barplot(
    xaxis_title='Split index',
    yaxis_title='Mean total return',
    legend_title_text='Symbol')

FigureWidget({
    'data': [{'name': 'BTC',
              'showlegend': True,
              'type': 'bar',
              'uid': '59e7b7db-4e18-4c6a-86de-fc86fcf98073',
              'x': array([0, 1]),
              'y': array([ 1.6226946 , -0.35382451])},
             {'name': 'ETH',
              'showlegend': True,
              'type': 'bar',
              'uid': '9b04af8a-9924-43bc-95cb-c7c5b5aa3770',
              'x': array([0, 1]),
              'y': array([ 0.65656512, -0.28316721])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'title': {'text': 'Symbol'},
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700,
               'xaxis': {'ti